In [ ]:
import os
%pwd

In [ ]:
# os.chdir("../")
%pwd

### Setup Dagshub repo(Mlflow remote repo)

- Add Github repo to dagshub
- Then configure following
    1. MLFLOW_TRACKING_URI
    2. MLFLOW_TRACKING_USERNAME
    3. MLFLOW_TRACKING_PASSWORD

In [63]:
import os 
from dotenv import load_dotenv

load_dotenv(".env")

username: str = os.getenv("MLFLOW_TRACKING_USERNAME")
mlflow_uri: str = os.getenv("MLFLOW_TRACKING_URI")
mlflow_pass: str = os.getenv("MLFLOW_TRACKING_PASSWORD")

# print(username, mlflow_uri, mlflow_pass)

In [64]:
from dataclasses import dataclass
from pathlib import Path

@dataclass
class ModelEvaluationConfig:
    root_dir: Path
    test_data_path: Path
    model_path: Path
    all_params: dict
    metrics_file_path: Path
    target_column: str
    mlflow_uri: str

In [65]:
from src.datascience.constants import *
from src.datascience.utils.common import read_yaml, create_directories,save_json
import os 
from dotenv import load_dotenv

In [70]:
class ConfiguratioManager:
    def __init__(
            self,
            config_filepath = CONFIG_FILE_PATH,
            params_filepath = PARAMS_FILE_PATH,
            schema_filepath = SCHEMA_FILE_PATH
            ):
        
        self.config= read_yaml(config_filepath)
        self.params= read_yaml(params_filepath)
        self.schema= read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])

    def get_model_evaluation_config(self) -> ModelEvaluationConfig:
        config = self.config.model_evaluation
        params = self.params.ElasticNet        
        schema= self.schema.TARGET_COLUMN
        
        create_directories([config.root_dir])

        model_evaluation_config = ModelEvaluationConfig(
            root_dir= config.root_dir,
            test_data_path= config.test_data_path,
            model_path= config.model_path,
            all_params= params,
            metrics_file_path= config.metrics_file_path,
            target_column= schema.name,
            mlflow_uri= os.getenv("MLFLOW_TRACKING_URI")            
        )

        return model_evaluation_config


In [71]:
import os
import pandas as pd
import numpy as np

from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import joblib

from urllib.parse import urlparse
import mlflow
import mlflow.sklearn

In [72]:
class ModelEvaluation:
    def __init__(self, config: ModelEvaluationConfig):
        self.config = config

    def eval_metrics(self, actual, pred):
        rmse = np.sqrt(mean_squared_error(actual, pred))
        mae = mean_absolute_error(actual, pred)
        r2 = r2_score(actual, pred) 
        return rmse, mae, r2
    
    def log_into_mlflow(self):
        test_data = pd.read_csv(self.config.test_data_path)
        model = joblib.load(self.config.model_path)

        test_X= test_data.drop([self.config.target_column], axis= 1)
        test_y= test_data[self.config.target_column]

        mlflow.set_registry_uri(self.config.mlflow_uri)
        tracking_url_type_store = urlparse(mlflow.get_tracking_uri()).scheme

        with mlflow.start_run():

            predicted_qualities = model.predict(test_X)

            (rmse, mae, r2)= self.eval_metrics(test_y, predicted_qualities)

            # Saving metrics as local
            scores = {
                "rmse": rmse,
                "mae": mae,
                "r2": r2
            }
            save_json(path= Path(self.config.metrics_file_path), data= scores)

            mlflow.log_params(self.config.all_params)

            mlflow.log_metric("rmse", rmse)
            mlflow.log_metric("r2", r2)
            mlflow.log_metric("mae", mae)

            # Model registery does not work with file store
            if tracking_url_type_store != "file":
                
                # Register the model
                # There are other ways to use the Model Registry, which depends on the use case,
                # please refer to the doc for more information:
                # https://mlflow.org/docs/latest/model-registry.html#api-workflow

                mlflow.sklearn.log_model(model, "model", registered_model_name="Wine-Quality-Pred-Elasticnet")
            else:
                mlflow.sklearn.log_model(model, "model")

In [73]:
try: 
    config= ConfiguratioManager()
    model_evaluation_config= config.get_model_evaluation_config()
    model_evaluation= ModelEvaluation(model_evaluation_config)
    model_evaluation.log_into_mlflow()
except Exception as e:
    raise e

[2024-12-31 23:07:48,599: INFO: common: yaml file: config\config.yaml loaded successfully]
[2024-12-31 23:07:48,602: INFO: common: yaml file: params.yaml loaded successfully]
[2024-12-31 23:07:48,603: INFO: common: yaml file: schema.yaml loaded successfully]
[2024-12-31 23:07:48,607: INFO: common: created directory at: artifacts]
[2024-12-31 23:07:48,609: INFO: common: created directory at: artifacts/model_evaluation]
[2024-12-31 23:07:49,088: INFO: common: json file saved at : artifacts\model_evaluation\metrics.json]


2024/12/31 23:08:01 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.
Successfully registered model 'Wine-Quality-Pred-Elasticnet'.
2024/12/31 23:08:07 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: Wine-Quality-Pred-Elasticnet, version 1
Created version '1' of model 'Wine-Quality-Pred-Elasticnet'.


🏃 View run righteous-steed-814 at: https://dagshub.com/25prathameshlohar/Wine-Quality-Prediction.mlflow/#/experiments/0/runs/2ee3d9e95eae4c308190ff58de1d76c1
🧪 View experiment at: https://dagshub.com/25prathameshlohar/Wine-Quality-Prediction.mlflow/#/experiments/0
